# Agentic Strategy Construction Tool
This is to produce trade reports that can be used in the evaluation section to backtest.

In [1]:
import copy
import pandas as pd
import requesters.company_data as cd
import agents.financial_agent as fa
import agents.committee_agent as ca
import importlib
from tqdm import tqdm
import json

from utils.s3_helper import S3Helper
from datetime import datetime

import concurrent.futures


In [2]:
importlib.reload(fa)
importlib.reload(ca)

<module 'agents.committee_agent' from '/project/agents/committee_agent.py'>

### Collect the data needed to run the Agents
The Agents will require financial statement datasets and news datasets in order to conduct their analysis.

In [3]:
# Get the company reports
security_datasets = cd.SecurityData('tmp/fs', 'dow_quarterly_ltm_v3.json')
# get the saved news datasets
s3_helper = S3Helper('tmp/fs')
s3_helper.get_file(filename='dow_headlines.parquet', local_filename='/tmp/dow_headlines.parquet')
# Convert to pandas dataframe
news_headlines = pd.read_parquet('/tmp/dow_headlines.parquet')

### Run the Agentic Models
Running the inference tasks across 893 date/ security combinations in 1:32h.

In [4]:
financial_agent = fa.FinancialAnalystAgent()
committee_agent = ca.CommitteeAgent()

In [5]:
def run_single(security: str, as_of_date: str) -> dict:
    """
    Function to run a single run of the Agent
    """
    company_data = security_datasets.get_security_all_data(as_of_date, security)
    # Time the run
    start_time = datetime.now()
    # Run the financial analyst agent
    financial_report = financial_agent.run(security_data=company_data, 
                                       news_data=news_headlines, 
                                       as_of_date=as_of_date)
    # Run the committee agent
    committee_report = committee_agent.run(senior_analyst_report=financial_report['senior_report'],
                                           financial_statement_analysis=financial_report['financial_report'],
                                           security_data=company_data)
    end_time = datetime.now()
    decision_dict = {
        'date': as_of_date,
        'security': security,
        'earning_decision': financial_report['final_output'].direction,
        'earning_magnitude': financial_report['final_output'].magnitude,
        'earning_confidence': financial_report['final_output'].confidence,
        'recommendation': committee_report['results'].recommendation,
        
        'responses': {'financial_analyst': financial_report,
                     'committee_report': committee_report},
        'time': str(end_time - start_time)
    }
    return decision_dict
    
    

In [6]:
dates_and_securities = security_datasets.date_security_timeseries()
test_security = dates_and_securities[20]

In [7]:
test_security

{'date': '2020-01-30', 'security': 'VZ UN Equity'}

In [12]:
%%time
test_output = run_single(test_security['security'],test_security['date'])

CPU times: user 223 ms, sys: 0 ns, total: 223 ms
Wall time: 2min 42s


In [13]:
#dict_keys(['date', 'security', 'earning_decision', 'earning_magnitude', 'earning_confidence', 'recommendation', 'responses', 'time'])
test_output['recommendation']

'HOLD'

### Run the backtest

In [8]:
data_output = []
def backtest(dates_and_securities):
    # create the securities
    progress = tqdm(total=len(dates_and_securities), position=0, leave=True)
    with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
        futures = [executor.submit(run_single, single['security'], single['date']) for single in dates_and_securities]
        for f in concurrent.futures.as_completed(futures):
            progress.update(n=1)
            data_output.append(f.result())
        

In [29]:
# Run the backtest
backtest(dates_and_securities[909:])

100%|██████████| 43/43 [07:59<00:00, 11.14s/it]  


In [24]:
len(data_output)

909

In [21]:
data_output[3]['recommendation']

'HOLD'

In [ ]:
print("hello")

### Save the raw output to S3

In [30]:
data_analysis_final = copy.deepcopy(data_output)

In [31]:
# Some of the items are not serializable. Convert to string before JSON
for item in data_analysis_final:
    item['responses']['financial_analyst']['final_output'] = str(item['responses']['financial_analyst']['final_output'])
    item['responses']['committee_report']['results'] = str(item['responses']['committee_report']['results'])
    item['responses']['committee_report']['history'] = str(item['responses']['committee_report']['history'])
    item['time']=str(item['time'])

In [32]:
with open('/tmp/agentic_output_llama_v5.json', 'w') as f:
    json.dump(data_analysis_final, f)

In [33]:
s3_helper.add_file(local_filename='/tmp/agentic_output_llama_v5.json')

### Create the Trade Report

In [36]:
def trade_report_generation(data, trade_only=False) -> pd.DataFrame:
    """
    Function to create the trade report to pass into the Strategy Analysis tool
    """
    trade_report = []
    if trade_only:
        for item in data:
            trade_report.append({'date': item['date'], 
                                 'security': item['security'], 
                                 'decision': item['recommendation'],
                                 'confidence': item['earning_confidence']})
    
        return pd.DataFrame(data=trade_report)
    else:
        for item in data:
            trade_report.append({'date': item['date'], 
                                 'security': item['security'], 
                                 'decision': item['recommendation'],
                                 'confidence': item['earning_confidence'],
                                 'earning_decision': item['earning_decision'],
                                 'earning_magnitude': item['earning_magnitude']})
        return trade_report

In [37]:
trades = trade_report_generation(data_analysis_final)

In [38]:
with open('Results/Agentic/trades_llama_v5.json', 'w') as f:
    json.dump(trades, f)

### Unused code

In [1]:
# from utils.s3_helper import S3Helper

# s3_helper = S3Helper('tmp/fs')
# s3_helper.get_file(filename='agentic_output_llama_v3_1.json',local_filename='/tmp/agentic_output_llama_v3_1.json')

In [2]:
# import json
# with open('/tmp/agentic_output_llama_v3_1.json', 'rb') as f:
#     data_output = json.load(f)

In [24]:
# print(data_output[1])

{'date': '2020-01-15', 'security': 'UNH UN Equity', 'earning_decision': 'increase', 'earning_magnitude': 'moderate', 'earning_confidence': 'moderate', 'recommendation': 'BUY', 'responses': {'financial_analyst': {'company_details': {'name': 'UnitedHealth Group Inc', 'figi_name': 'BBG000CH5208', 'sector': 'Health Care', 'sec_fs': 'Income Statement:                                                        t           t-1           t-2           t-3           t-4           t-5\nitems                                                                                                                          \nRevenue                                      2.421550e+11  2.396710e+11  2.358760e+11  2.313670e+11  2.262470e+11  2.198910e+11\nSales and Services Revenues                  5.057000e+10  5.087100e+10  4.994400e+10  4.836800e+10  4.678400e+10  4.502500e+10\nOther Revenue                                1.915850e+11  1.888000e+11  1.859320e+11  1.829990e+11  1.794630e+11  1.748660e+11\nCost of

In [20]:
len(data_analysis_final)

893

In [10]:
len(data_output)

219